In [1]:
from pickle import load, dump
from os import makedirs
from os.path import dirname

import pandas as pd

In [25]:
   
def predict_data(data, model, ml_model: str, lags, **kwargs):
    if ml_model[0: 3] == 'svr' or ml_model[0: 3] == 'mlp' or ml_model[0:2] == 'rf':
        data = data.testing_set
        return model.predict(data[:, lags])
    elif ml_model[0: 7] == 'xgboost':
        data = data.testing_set

        return model.predict(data[:, lags])
    elif ml_model[0: 4] == 'lstm':
        data = data.testing_set
        data = data[:, lags]
        data = data.reshape((data.shape[0], data.shape[1], 1))
        return model.predict(data).ravel()
    elif ml_model[0: 3] in ['dee', 'tft']:
        data = data.testing_x
        pred = []

        for d in data:
            predictions = model.predict(d)

            for x in predictions:
                pred.append(x.mean[0])

        return pred

    elif ml_model[0: 6] == 'da-rnn':
        data = data.testing_x
        import torch

        with torch.no_grad():
            a = model(data.cuda())

        return list(a.cpu().repeat(1, 2).numpy()[:, -1])

    elif ml_model[0: 5] == 'arima':
        data = data.testing_set
        arima_forecast = kwargs.get('arima_forecast')
        if arima_forecast:
            if arima_forecast == 'in_sample':
                return model.predict_in_sample(data)[data]
            elif arima_forecast == 'out_sample':
                return model.predict(data + 1)[-1]
        else:
            return model.predict(len(data))

In [19]:
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
import numpy as np

lags = [10]#, 20, 30, 40, 50, 60]
modelos = ['arima', 'da-rnn', 'deep-ar', 'deep-state', 'lstm', 'tft', 'mlp', 'rf', 'svr', 'xgboost']

for metrica in ['traffic']:
    for index in range(1, 11):
        parameters = []

        for modelo in modelos:            
            for lag in lags:
                # Open the pickle file and load the model manually using torch.load
                with open('pickle/microservice '+str(index)+'/'+metrica+'/hyper_parameter/'+modelo+str(lag)+'.pkl', 'rb') as f:
                    model = load(f)  # Load the object
                
                # Now manually load the PyTorch model onto the CPU
                if isinstance(model, torch.nn.Module):
                    model = torch.load(f, map_location=torch.device('cpu'))

                #print()
                parameters.append(model['time_to_fit'])
                
                # if modelo == "lstm":
                #     model.summary(print_fn=lambda x: summary_str.write(x + "\n"))
                #     summary_str_value = summary_str.getvalue()
                #     parameters.append(summary_str_value)
                #     #parameters.append(print(model.to_json()))
                # elif modelo == "xgboost":
                #     parameters.append(model.__dict__) 
                # elif modelo == "deep-state":
                #     parameters.append(model.__dict__)
                # elif modelo == "deep-ar":
                #     parameters.append(model.__dict__)
                # elif modelo == "da-rnn":
                #     parameters.append(model.__dict__)
                # elif modelo == "tft":
                #     parameters.append(model.__dict__)
                # else:
                #     parameters.append(model)
                    
                #print(predictions[modelo][lag])
        
        #dump(df, open('forecasting/microservice '+str(index)+'/'+metrica+'.pkl', 'wb'))
        result_df = pd.DataFrame(data=np.array(parameters).reshape(-1, len(lags)), index=modelos, columns=['Time to Fit (s)'])
        #print(result_df)
        #result_df.to_csv('costs.csv')
        result_df.to_csv('other-results/costs/microservice '+str(index)+'/'+metrica+'.csv')

/home/wellison/Development/PycharmProjects/comparative-analysis-microservices-prediction/venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
[19:58:58] src/nnvm/legacy_json_util.cc:204: Warning: loading symbol saved by MXNet version 10901 with lower version of MXNet v10600. May cause undefined behavior. Please update MXNet if you encounter any issue
[19:58:58] src/nnvm/legacy_json_util.cc:204: Warning: loading symbol saved by MXNet version 10901 with lower version of MXNet v10600. May cause undefined behavior. Please update MXNet if you encounter any issue
[19:58:58] src/nnvm/legacy_json_util.cc:204: Warning: loading symbol saved by MXNet version 10901 with lower version of 